In [20]:
import psycopg2
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from config import config

zona = 'tornavacas'

params = config()
conn = psycopg2.connect(**params)
    
# SUPERFICIE

coordenadas = pd.read_sql('''
            SELECT nombre, 
                ST_X(ST_Centroid(ST_Transform(geom, 25830))) as coordenada_x, 
                ST_Y(ST_Centroid(ST_Transform(geom, 25830))) as coordenada_y
            FROM quema_tornavacas_todas
            ORDER BY parcela
                ''', conn)
coordenadas.columns = ['Parcela', 'Coordenada X', 'Coordenada Y']
display(coordenadas)

sup_df = pd.read_sql('''
            SELECT
                nombre, ST_Area(ST_Transform(geom, 25830)) / 10000 as sup
            FROM quema_tornavacas_todas
            ORDER by parcela 
            ''', conn)
sup_df.columns = ['Parcela', 'Superficie (ha)']

# #PENDIENTES Y ALTITUD

stats = pd.read_sql('''
            SELECT a.nombre,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(a.geom, 25830))))).min as altitud_min,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(a.geom, 25830))))).max as altitud_max,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(a.geom, 25830))))).mean as altitud_media,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(a.geom, 25830))))).mean as pendiente
            from quema_tornavacas_todas a, pendientes25 b, mdt25 c
            GROUP BY a.nombre, a.parcela
            ORDER BY a.parcela
            ''', conn)

stats.columns = ['Parcela', 'Altitud mínima', 'Altitud máxima', 'Altitud media', 'Pendiente']

# # ORIENTACIONES

orientaciones_df = pd.read_sql('''
            SELECT a.nombre,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(a.geom, 25830))))).mean as orientacion
            FROM quema_tornavacas_todas a, orientaciones25 b
            GROUP BY a.nombre, a.parcela 
            ORDER BY a.parcela
            ''', conn)    

orientaciones_df.columns = ['Parcela', 'Orientación']

# # HABITATS

# habitats_parcelas = pd.read_sql('''
#             WITH ab as (
#                 SELECT
#                     a.nombre,
#                     codue as codigo,
#                     descripcio, 
#                     st_area(st_transform(a.geom, 25830)) / 10000 as sup_parcela,
#                     st_area(st_intersection(b.geom, st_transform(a.geom, 23030))) / 10000 as superficie,
#                     st_area(st_intersection(b.geom, st_transform(a.geom, 23030))) / st_area(st_transform(a.geom, 23030)) * 100 as porcentaje
#                 FROM atlashabitats2005_extremadura b, quema_tornavacas_todas a
#                 WHERE st_intersects(st_transform(a.geom, 23030), b.geom)
#                 GROUP BY a.nombre, b.codue, b. descripcio, a.parcela, b.geom, a.geom
#                 ORDER BY a.parcela
#             )
#             SELECT 
#                 ab.*,
#                 Sum(St_Area(St_Intersection(atlas.geom, St_Transform(zec.geom, 23030)))) / 10000 as sup_hab_zec,
#                 ab.superficie * 100 / Sum(St_Area(St_Intersection(atlas.geom, St_Transform(zec.geom, 23030)))) / 10000 as sup_hab_zec
#             FROM ab, atlashabitats2005_extremadura atlas, sierra_gredos_y_valle_jerte zec
#             WHERE st_intersects(atlas.geom, st_transform(zec.geom, 23030)) and ab.codigo = atlas.codue and ab.descripcio = atlas.descripcio
#             GROUP BY ab.nombre, ab.codigo, ab.descripcio, ab.sup_parcela, ab.superficie, ab.porcentaje
#             ''', conn)

# habitats_parcelas.columns = ['Parcela', 'Código UE', 'Descripción', 'Superficie Parcela', 'Superficie del hábitat en la parcela', 'Porcentaje de parcela ocupado por el hábitat', 'Superficie del hábitat en el ZEC', 'Porcentaje de la superficie del hábitat en la parcela sobre el total del hábitat en el ZEC']
display(habitats_parcelas)
    
prescripcion = pd.DataFrame({
    'Factores meteorológicos': ['Temperatura', 'Humedad Relativa', 'Viento', 'Humedad del suelo', 'HCFM', 'Nº días sin llover'],
    'Rango base': ['5-20 ºC (heladas previas)', '30-60%', '5-15 km/h', '50-100%', '5-11%', '5-25 días'],
    'Rango práctico': ['6-21 ºC', '27 - 64 %%', '0-25 km/h', 'Húmedo', '6-10%', '2-12 días']
})

# FICHAS

parcelas = pd.read_sql('''
            SELECT
                a.nombre,
                a.parcela,
                ST_Area(ST_Transform(a.geom, 25830)) / 10000 as sup, 
                ST_Perimeter(ST_Transform(a.geom, 25830)) as perim,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).min as pend_min,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).max as pend_max,
                (ST_SummaryStats(St_Union(ST_Clip(b.rast, st_transform(geom, 25830))))).mean as pend_mean,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).min as alt_min,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).max as alt_max,
                (ST_SummaryStats(St_Union(ST_Clip(c.rast, st_transform(geom, 25830))))).mean as alt_mean,
                (ST_SummaryStats(St_Union(ST_Clip(d.rast, st_transform(geom, 25830))))).mean as orient
            FROM 
                quema_tornavacas_todas a,
                pendientes25 b,
                mdt25 c,
                orientaciones25 d
            GROUP BY
                a.nombre, a.parcela, a.geom
            ''', conn)
display(parcelas.to_dict(orient='records'))
    
conn.close()

env = Environment(loader=FileSystemLoader('static/plan_quemas'))
template =env.get_template('base.html')
output = template.render(
    coordenadas = coordenadas,
    zona = zona,
    sup = sup_df,
    stats = stats,
    orientaciones = orientaciones_df,
    prescripcion = prescripcion,
    habitats = habitats_parcelas,
    parcelas = parcelas.to_dict(orient='records')
)

with open('static/plan_quemas/plan_quemas.html', 'w') as f:
    f.write(output)

,Parcela,Coordenada X,Coordenada Y
0,Parcela 1,271189.476228,4.463587e+06
1,Parcela 2,271265.711385,4.463475e+06
2,Parcela 3,271360.793499,4.463330e+06
3,Parcela 4,272179.317981,4.462472e+06
4,Parcela 5,272237.786929,4.462282e+06
5,Parcela 6,272157.331568,4.462194e+06
6,Parcela 7,271753.942183,4.462426e+06
7,Parcela 8,271463.435117,4.462341e+06
8,Parcela 9,271037.553146,4.462251e+06
9,Parcela 10,270274.597087,4.461993e+06


,Parcela,Código UE,Descripción,Superficie Parcela,Superficie del hábitat en la parcela,Porcentaje de parcela ocupado por el hábitat,Superficie del hábitat en el ZEC,Porcentaje de la superficie del hábitat en la parcela sobre el total del hábitat en el ZEC
0,Parcela 1,5120,Formaciones montanas de Genista purgans,0.499272,0.486953,97.530310,27016.387708,1.802435e-11
1,Parcela 10,5120,Formaciones montanas de Genista purgans,3.446293,3.446371,100.000000,27016.387708,1.275660e-10
2,Parcela 2,5120,Formaciones montanas de Genista purgans,0.392823,0.392832,100.000000,27016.387708,1.454051e-11
3,Parcela 3,5120,Formaciones montanas de Genista purgans,0.662955,0.660062,99.561442,27016.387708,2.443192e-11
4,Parcela 4,5120,Formaciones montanas de Genista purgans,0.797826,0.797844,100.000000,27016.387708,2.953184e-11
5,Parcela 5,5120,Formaciones montanas de Genista purgans,7.901258,7.901437,100.000000,27016.387708,2.924683e-10
6,Parcela 6,4090,Brezales oromediterráneos endémicos con aliaga,3.777422,0.146860,3.887744,26005.488803,5.647263e-12
7,Parcela 6,5120,Formaciones montanas de Genista purgans,3.777422,3.630648,96.112256,27016.387708,1.343869e-10
8,Parcela 6,9230,Robledales galaico-portugueses con Quercus rob...,3.777422,0.146860,3.887744,21049.584193,6.976852e-12
9,Parcela 7,5120,Formaciones montanas de Genista purgans,0.598629,0.598642,100.000000,27016.387708,2.215849e-11


[{'nombre': 'Parcela 1',
  'parcela': 1,
  'sup': 0.49927223895004963,
  'perim': 348.1820142831095,
  'pend_min': 20.347434997558594,
  'pend_max': 39.20896530151367,
  'pend_mean': 28.90689550127302,
  'alt_min': 1994.498046875,
  'alt_max': 2024.530029296875,
  'alt_mean': 2008.7734375,
  'orient': 128.8893530709403},
 {'nombre': 'Parcela 10',
  'parcela': 10,
  'sup': 3.446293488113699,
  'perim': 1579.5426661866372,
  'pend_min': 19.980243682861328,
  'pend_max': 50.993408203125,
  'pend_mean': 35.490500612078975,
  'alt_min': 1574.52099609375,
  'alt_max': 1622.623046875,
  'alt_mean': 1591.0523911961968,
  'orient': 169.3145104174344},
 {'nombre': 'Parcela 2',
  'parcela': 2,
  'sup': 0.3928232822242626,
  'perim': 340.71462931145703,
  'pend_min': 21.92401123046875,
  'pend_max': 32.69502258300781,
  'pend_mean': 28.188835779825848,
  'alt_min': 1961.708984375,
  'alt_max': 1987.4000244140625,
  'alt_mean': 1974.6700032552083,
  'orient': 135.7542241414388},
 {'nombre': 'Parcel